In [ ]:
import setup
from imports import *
from scripts.analyze_stock import *  
from scripts.scrape_url import *
from scripts.plotting_routines import *

day = '2024-07-03'
date = datetime.strptime(day, '%Y-%m-%d')

# Set the time frame
end_date = date
start_date = date - timedelta(days=28)

ticker_data_list = []
base_url = 'https://finance.yahoo.com/calendar/earnings/?day=' + day

offset = 0
while True:
    url = f"{base_url}&offset={offset}&size=100"
    print(url)
    table = extract_table(url)
    
    if table is None:
        break

    ticker_data_list = process_earnings_table(table, ticker_data_list)
    offset += 100  # Increment the offset by 100 for the next iteration

ticker_data_sorted = convert_to_dataframe(ticker_data_list)

filtered_tickers = []

# Iterate through the sorted list of tickers
for index, row in ticker_data_sorted.iterrows():
    ticker = row['Symbol']
    
    if ticker in filtered_tickers:
        continue

    # Use the function from utils.py to analyze stock options
    options_metrics = analyze_stock_options(ticker)
    
    if options_metrics['total_itm_calls'] == 0 and options_metrics['total_itm_puts'] == 0:
        continue

    # Get All Analysis
    print_options_data(ticker, options_metrics)

    # Call the plot_stock_history method
    plot_stock_history(ticker, start_date, end_date)

    # IV Skew
    plot_iv_skew_otm_only(options_metrics, end_date, ticker)

    # Sector ETF Time Series
    plot_etf_historical_data(ticker, start_date, end_date)

    # Filtered tickers
    filtered_tickers.append(ticker)
    
del filtered_tickers, ticker_data_list, ticker_data_sorted


